# <span style="color:lightcoral; font-family:freestyle script; font-size:4em">SLOSH till relationsdatabas</span> <br><span style="color:black; font-family:freestyle script; font-size:3em">Förbereda färdig metadata för SQLite</span>
***

# Innehållsförteckning  
1. [Introduktion](#1)   
2. [Tabeller](#2)  
    2.1 [Variabler](#2.1)  
    2.2 [Frekvenstabell](#2.2)  
    2.3 [Enkättexter](#2.3)  
    2.4 [Kodlistor](#2.4)  
    2.5 [Svarstexter](#2.5)  
    2.6 [Kodlistor_stor](#2.6)
3. [Exportera csv-filer](#3)

***

# <a id = "1">1. Introduktion</a>  
I vad som följer kommer kursiv stil endast användas för att benämna *tabeller*. Fetstil kommer endast användas för att benämna **kolumner**, och värden i en kolumn kommer att vara i stil med (t.ex.) `SLOSHW08`, `Missing`, `142`. Även datatyper (t.ex. `str` eller `float64`) och pythonobjekt (t.ex. listor) kommer att ha denna formatering. Den metadata som redan är dokumenterad enligt Vetenskapsrådets riktlinjer ligger i excelfilen *metadata*. Några av tabellerna i relationsdatabsen innehåller kolumner som inte finns i denna fil. Dessa kolumner skapas allt eftersom. Därefter kommer följande tabeller att skapas:  

|Variabler|Enkättexter|Kodlistor|Svarstexter|Kodlistor_stor|  
|:---:|---|---|---|---|
|Variabel|Enkättext|Kodlista|Svarstext|K_id|  
|Enkät|Källa|From|From|Kodlista|  
|Beskrivning|-|Tom|Tom|Kod|  
|Itemnr|-|-|-|Svarstext|  
|Enkättext|-|-|-|-|  
|Kodlista|-|-|-|-|  
|Dubbelkodning|-|-|-|-|
  
  
Relationsdatabasen kommmer att innehålla ytterligare fem tabeller. Dessa är *Frekvenstabell*, *Begrepp*, *Begrepp_stor*, *Tidsserie* samt *Tidsserie_stor*. För tillfället finns ingen data till dessa tabeller i *metadata* och därför kommer de inte skapas här. Det finns dock data för *Frekvenstabell* men den är inte tillförlitlig och går snabbt att återskapa från SLOSH-datafilen.  

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Metadata.xlsx')

För att ta fram *Kodlista_stor* används `missing` i **text**. I *metadata* stavas `Missing` ibland med "M" och ibland med "m"(`missing`). Python läser detta som olika `str`, därför ersätts liten bokstav med stor.

In [3]:
df.replace('missing', 'Missing', inplace = True)

Senare kommer metoden `groupby.min()` användas för att välja du exempelvis kodlistor och svarstexter för vilka **Tom**-värdet är lägsta möjliga. Det förekommer `NaN` i **Text** vilket tyvärr inte betraktas som en kategori av `groupby.min()`. Därför ersätt detta värde med `-` i **Text**.

In [4]:
df.fillna({'Text': '-'}, inplace = True)

För närvarande är kodlistorna strängar med "pedagogiska" namn som uppkommit under dokumentationsprocessen. Det vore enklare ifall dessa namn ersattes av heltal. Anledningen är att framgent, när resten av SLOSH ska dokumenteras, blir arbetsflödet mindre komplicerat ifall kodlistorna representeras av heltal.

In [5]:
# Unika kodlistor
unikod = list(df.Kodlista.unique())

# Ett heltal för varhe kodlista
siffror = range(len(unikod))

# En dictionary som parar ihop varje kodlista med en siffra
dict = {unikod[i]:siffror[i] +1 for i in siffror}

# dict används för att ersätta kodlistorna med siffror
df.Kodlista.replace(dict, inplace = True)

In [6]:
df.columns

Index(['Namn', 'Tidsserie', 'Beskrivning', 'Enkät', 'Dubbelkodning',
       'Enkättext', 'From', 'Tom', 'Kodlista', 'From.1', 'Tom.1', 'Kod',
       'Text', 'From.2', 'Tom.2', 'Koncept 1', 'Koncept 2', 'Instrument 1',
       'Instrument 2', 'Källa', 'SLOSHW06', 'frek', 'SLOSHW08', 'frek.1',
       'SLOSHW10', 'frek.2', 'SLOSHW12', 'frek.3', 'SLOSHW14', 'frek.4',
       'SLOSHW16', 'frek.5', 'SLOSHW18', 'frek.6', 'SLOSHNW06', 'frek.7',
       'SLOSHNW08', 'frek.8', 'SLOSHNW10', 'frek.9', 'SLOSHNW12', 'frek.10',
       'SLOSHNW14', 'frek.11', 'SLOSHNW16', 'frek.12', 'SLOSHNW18', 'frek.13'],
      dtype='object')

***  
# <a id = "2">2. Tabeller</a>  
I det här avnittet kommer tabellerna (förutom de som rör tidsserier och begrepp, se [introduktionen](#1)) till relationsdatabasen att skapas. Först tabellen *Variabler* (som är störst). Därefter *Enkättexter*, *Kodlistor*, *Svarstexter* och till sist *Kodlistor_stor*.

***  
## <a id = "2.1">2.1 Variabler</a>  
Den här tabellen ska innehålla kolumnerna **Variabel**, **Enkät**, **Beskrivning**, **Dubbelkodning**, **Itemnr**, **Enkättext**  och **Kodlista**. Samtliga kolumner finns redan i *metadata* (numera `df`), dock kallas **Variabel** där för **Namn** och måste därför döpas om. Även **Enkät** måste ändras. Istället för `working` ska det stå t.ex. `SLOSH 06 working` (på samma sätt för `non-working`. Ifall värdet är `w+n` ska variabelns första förekomst ha ett `working`-värde medan det andra ska ha ett `non-working`-värde. Detta eftersom excelfil är sådan att `working`-version av en variabel, t.ex. `promoted_4`, alltid kommer före `non-working`-versionen av densamma. 

In [7]:
Variabler = df.loc[:, ['Namn', 'Enkät', 'Beskrivning', 'Dubbelkodning', 'Enkättext', 'Kodlista',
               'SLOSHW06', 'frek', 'SLOSHW08', 'frek.1', 'SLOSHW10', 'frek.2', 'SLOSHW12', 'frek.3', 
                'SLOSHW14', 'frek.4', 'SLOSHW16', 'frek.5', 'SLOSHW18', 'frek.6', 'SLOSHNW06', 'frek.7',
               'SLOSHNW08', 'frek.8', 'SLOSHNW10', 'frek.9', 'SLOSHNW12', 'frek.10',
               'SLOSHNW14', 'frek.11', 'SLOSHNW16', 'frek.12', 'SLOSHNW18', 'frek.13']]

Variabler = Variabler.rename({'Namn': 'Variabel'}, axis = 'columns')

Variabler som är dubbelkodade har enkätvärdet `w+n`. Dubbelkodning innebär att de inte är helt identiska, antingen med avseende på kodlista eller enkättext. Här ersätts `w+n` med `working` (första förekomst per variabel) och `non-working` (andra förekomst per variabel).

In [8]:
for i in Variabler.index:
    
    # om dubbelkodning = 1
    if Variabler.iloc[i, 2] == 1:
        
        # ändra enkätvärdet till "working"
        Variabler.iloc[i, 1] = 'working'
        
        # ändra enkätvärde för nästa variabel till non-working (eftersom working alltid ligger före n-working)
        Variabler.iloc[i+1, 1] = 'non-working'
        
        # För att nästkommande variabel ska undantas måste dess värde i dubbelkodning ändras från 1 till ngt annat
        Variabler.iloc[i+1, 2] = 0

De variabler som inte är dubbelkodade måste identifieras. En dataframe som sträcker sig fram till varje sådan rad kommer att skäras ut tillsammans med raden ifråga. Detta läggs i en lista som sedan sammanfogas till en ny dataframe. På så vis "dubbleras" varje variabel som är `w+n` men samtidigt inte dubbelkodad.

In [9]:
# Listan som alla delar läggs i.
a = []

# startpunkten för alla snitt som ska göras. Startunkten uppdateras i loopen.
b = 0

# gå igenom varje rad
for i in Variabler.index:
    
    # identifiera "w+n"
    if Variabler.iloc[i, 1] == 'w+n':
        
        # sätt enkätvärdet till "working"
        Variabler.iloc[i, 1] = 'working'
        
        # skär ut en dataframe
        a.append(Variabler.iloc[b:i+1, :])
        
        # gör raden till en series
        c = Variabler.iloc[i, ]
        
        # gör om till en dataframe och transponera
        c = c.to_frame().transpose()
        
        # sätt enkätvärdet till "non-working"
        c['Enkät'] = 'non-working'
        
        # lägg till i listan med delar       
        a.append(c)
        # uppdatera startpunkten för nästa dataframe som ska skäras ut
        b = i+1

# sammanfoga alla delar
Variabler = pd.concat(a, ignore_index = True)
    

Nu ersätts enkätvärden `working` och `non-working` med vågspecifika motsvarigheter. För att åstadkomma detta kan den sista symbolen i varje variabelnamn, som alltid är en siffra mellan 1-7, omvandlas till det årtal då data samlats in. Variabeln promoted_4 samlades in 2012 och ska således ha enkätvärdet `SLOSH 12 working` (p.s.s. non-working). Låt siffrorna 1, 2, ..., 7 stå för vågornas ordningsföljd. Årtalen är 6, 8,..., 18. Sambandet mellan dessa talföljder ges således av  
  
  $ År = 3*(våg-1)-(våg-1)$ 

In [10]:
for i in Variabler.index:
    
    # radens enkätvärde
    enkät = Variabler.iloc[i, 1]
    
    # radens namnvärde
    namn = Variabler.iloc[i, 0]
    
    # vågnummret ges av sista symbolen i namnet.
    våg = int(namn[-1])
    
    # Årtal kan sedan räknas ut enligt sambandet ovan.
    år = str(3*(våg+1) - (våg - 1))
    
    # Ifall årtalet är ett ental ska det föregås av 0
    if len(år) == 1:
        år = '0{}'.format(år)
    
    # Översättning av working
    if enkät == 'working':
        Variabler.iloc[i, 1] = 'SLOSH {} working'.format(år)
    
    # Översättning av non-working
    if enkät == 'non-working':
        Variabler.iloc[i, 1] = 'SLOSH {} non-working'.format(år)

Notera att 1 och 0 är kvar i kolumnen **Dubbelkodning**.

Nu ska kolumnen **Itemnr** skapas. Först två listor med "working"- respektive "non-working"-kolumner skapas.

In [11]:
WORKING = ['SLOSHW06', 'SLOSHW08', 'SLOSHW10', 'SLOSHW12', 'SLOSHW14', 'SLOSHW16', 'SLOSHW18']

NONWORKING = ['SLOSHNW06', 'SLOSHNW08', 'SLOSHNW10', 'SLOSHNW12', 'SLOSHNW14', 'SLOSHNW16', 'SLOSHNW18']

Skapa en ny kolumn, kalla den för "Itemnr" och fyll den med `NaN`.

In [12]:
Variabler['Itemnr'] = np.nan

I dessa kolumner förekommer ett itemnr ifall en given variabel hör till vågen ifråga. I annat fall är det `NaN`. Varje rad har värden i högst en vågkolumn (t.ex. **SLOSHW06**). Detta värde kopieras och läggs i **Itemnr**. 

In [13]:
for w, nw in zip(WORKING, NONWORKING):
    
    Variabler.loc[Variabler[w].isna() == False, ['Itemnr']] = Variabler[w]
    Variabler.loc[Variabler[nw].isna() == False, ['Itemnr']] = Variabler[nw]

Ta bort alla överflödiga kolumner.

In [14]:
Variabler.drop(Variabler.columns[6:34], axis = 1, inplace = True)

Varje par av **Variabel** och **Enkät** ska endast förekomma en gång i *Variabler*. Skär ut dessa två kolumner och hitta den första förekomsten av varje par och lägg tillhörande index i en lista.

In [15]:
behåll = list(Variabler.iloc[:, :2].drop_duplicates().index)

Nu skapas komplementet till `behåll`.

In [16]:
kasta = [i for i in Variabler.index if i not in behåll]

Här kastas alla rader som förekommer i komplementet till `behåll`.

In [17]:
Variabler.drop(kasta, inplace = True)

Index är måste återställas.

In [18]:
Variabler.set_index('Variabel', inplace = True)

Ändra ordningen på kolumnerna.

In [19]:
Variabler = Variabler[['Enkät', 'Beskrivning', 'Dubbelkodning', 'Itemnr', 'Enkättext', 'Kodlista']]

In [20]:
Variabler.head()

,Enkät,Beskrivning,Dubbelkodning,Itemnr,Enkättext,Kodlista
Variabel,,,,,,
promoted_1,SLOSH 06 working,NaN,NaN,A1_a,Ditt arbete @ Har du befordrats/fått en högre ...,1
promoted_2,SLOSH 08 working,NaN,NaN,4,Har du fått en annan tjänsteposition de senast...,2
promoted_3,SLOSH 10 working,NaN,NaN,8,Har du fått en förändrad tjänsteposition de se...,3
promoted_4,SLOSH 12 working,NaN,1,4,Har du fått en förändrad tjänsteposition under...,3
promoted_4,SLOSH 12 non-working,NaN,1,4,Har du fått en förändrad tjänsteposition under...,3


***  
## <a id = "2.3">2.3 Enkättexter</a>  
Består endast av **Enkättext** och **Källa**. Eventuellt bör **Enkättext** göras till index eftersom den är huvudnyckel i tabellen.

In [21]:
Enkättexter = df.loc[:, ['Enkättext', 'Källa']]

In [22]:
Enkättexter = Enkättexter.drop_duplicates().set_index('Enkättext')

In [23]:
Enkättexter.head()

,Källa
Enkättext,
Ditt arbete @ Har du befordrats/fått en högre tjänst under de två senaste åren?,NaN
Har du fått en annan tjänsteposition de senaste två åren?,NaN
Har du fått en förändrad tjänsteposition de senaste två åren?,NaN
Har du fått en förändrad tjänsteposition under de senaste två åren?,NaN
Har du fått en sämre position under de senaste två åren?,NaN


***  
## <a id = "2.4">2.4 Kodlistor</a>  
Består av **Kodlista** (nyckel) samt **From.1** och **Tom.1**. Notera att de sista klolumnerna blivit omdöpta automatiskt vid inläsning av *metadata*. 

In [24]:
Kodlistor = df.loc[:, ['Kodlista', 'From.1', 'Tom.1']]

Varje kodlista ska endast förekomma en gång.

In [25]:
Kodlistor = Kodlistor.drop_duplicates().reset_index(drop = True)

Samma kodlista kan fortfarande förekomma flera gånger eftersom **From.1** har fyllts i flera gånger. Tanken var från början att detta skulle ordnas i efterhand i excel. Behåll endast de rader som har lägsta möjliga värde i **From.1** för varje värde i **Kodlista**.

In [26]:
Kodlistor = Kodlistor.groupby('Kodlista').min()

In [27]:
Kodlistor.head()

,From.1,Tom.1
Kodlista,,
1,2006,NaN
2,2008,NaN
3,2010,NaN
4,2014,NaN
5,2016,NaN


***  
## <a id = "2.5">2.5 Svarstexter</a>  
Består av **Svarstext** (nyckel) samt **From** och **Tom**. Den första kolumnen heter **Text** i *metadata* och döps därför om. 

In [28]:
Svarstexter = df.loc[:, ['Text', 'From', 'Tom']]
Svarstexter.rename(columns = {'Text':'Svarstext'}, inplace = True)

Varje svarstext ska endast förekomma en gång.

In [29]:
Svarstexter.drop_duplicates(inplace = True)

Se kommentar ovan i **2.4**.

In [30]:
Svarstexter = Svarstexter.groupby('Svarstext').min()

In [31]:
Svarstexter.head()

,From,Tom
Svarstext,,
0,2006,NaN
-,2006,NaN
... mycket överkvalificerad,2006,NaN
... skulle behöva en hel del ytterligare kunskaper,2006,NaN
... skulle behöva ytterligare vissa kunskaper,2006,NaN


***  
## <a id = "2.6">2.6 Kodlistor_stor</a>  
Den här tabellen ska innehålla kodlistornas namn i kombination med deras kod och svarstext. Varje sådan kombination ska endast förekomma en gång. Börja med att skära ut kolumnerna **Kodlista**, **Kod** och **Text** ur *metadata*.

In [32]:
Kodlistor_stor = df.loc[:, ['Kodlista', 'Kod', 'Text']]  

Döp om kolumnen med svarstexter.

In [33]:
Kodlistor_stor.rename(columns = {'Text':'Svarstext'}, inplace = True)

Behåll endast unika rader så att varje kodlista endast förekommer en gång.

In [34]:
Kodlistor_stor.drop_duplicates(inplace = True)

Använd kolumnen **Kodlista** som index.

In [35]:
Kodlistor_stor.set_index('Kodlista', inplace = True)

In [36]:
Kodlistor_stor.head(20)

,Kod,Svarstext
Kodlista,,
1,1,Ja
1,2,Nej
1,.,Missing
2,1,Mycket högre
2,2,Något högre
2,3,Oförändrad
2,4,Något lägre
2,5,Mycket lägre
2,.,Missing


***  
# <a id = "3">3. Exportera csv-filer</a>  
Nu ska samtliga tabeller exporteras till csv-filer för att läggas in i SQLite.

In [37]:
Variabler.to_csv('Variabler.csv', encoding = 'UTF-8')

Enkättexter.to_csv('Enkättexter.csv', encoding = 'UTF-8')

Kodlistor.to_csv('Kodlistor.csv', encoding = 'UTF-8')

Kodlistor_stor.to_csv('Kodlistor_stor.csv', encoding = 'UTF-8')

Svarstexter.to_csv('Svarstexter.csv', encoding = 'UTF-8')
